In [106]:
import pandas as pd
import requests
import re
import time
from bs4 import BeautifulSoup
from tqdm import tqdm


## Load CSV File

In [107]:
df = pd.read_csv('../data/RC_books.csv' , encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36358 entries, 0 to 36357
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Acc. Date           36179 non-null  object 
 1   Acc. No.            36358 non-null  int64  
 2   Title               36358 non-null  object 
 3   ISBN                36358 non-null  object 
 4   Author/Editor       36295 non-null  object 
 5   Ed./Vol.            5779 non-null   object 
 6   Place & Publisher   36358 non-null  object 
 7   Year                36188 non-null  float64
 8   Page(s)             36357 non-null  object 
 9   Class No./Book No.  36358 non-null  object 
 10  Unnamed: 10         23 non-null     object 
 11  Unnamed: 11         20 non-null     object 
 12  Unnamed: 12         13 non-null     object 
 13  Unnamed: 13         13 non-null     object 
 14  Unnamed: 14         11 non-null     object 
 15  Unnamed: 15         10 non-null     object 
 16  Unna

C:\Users\Gaurang\AppData\Local\Temp\ipykernel_22804\2652610581.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/RC_books.csv' , encoding='latin-1')


## Remove Unnamed Columns

In [108]:

df_unnamed = df[['Unnamed: 10', 'Unnamed: 11','Unnamed: 12' , 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']]
df = df.drop(columns= df_unnamed)

## Remove Duplicate Data

In [109]:
df = df.drop_duplicates(subset=['ISBN', 'Title' , 'Author/Editor'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32012 entries, 0 to 36357
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Acc. Date           31841 non-null  object 
 1   Acc. No.            32012 non-null  int64  
 2   Title               32012 non-null  object 
 3   ISBN                32012 non-null  object 
 4   Author/Editor       31981 non-null  object 
 5   Ed./Vol.            3723 non-null   object 
 6   Place & Publisher   32012 non-null  object 
 7   Year                31846 non-null  float64
 8   Page(s)             32011 non-null  object 
 9   Class No./Book No.  32012 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.7+ MB


## Used OpenLibrary API to fetch the Data

In [ ]:

df = df[:].copy()
df.reset_index(drop=True, inplace=True)

if "book_description" not in df.columns:
    df["book_description"] = None

OUTPUT_CSV = "../CSV_Files/description.csv"

ISBN_COL = "ISBN"
TITLE_COL = "Title"
AUTHOR_COL = "Author/Editor"

DESC_COL = "description"
SOURCE_COL = "source"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

SLEEP_TIME = 0.3
SAVE_EVERY = 200
TIMEOUT = 10

# =========================
# CLEANING FUNCTIONS
# =========================
def clean_isbn(isbn):
    if not isbn or pd.isna(isbn):
        return None
    isbn = re.sub(r"[^0-9Xx]", "", str(isbn))
    return isbn if len(isbn) in [10, 13] else None


def clean_text(text):
    if not text or pd.isna(text):
        return None

    text = text.lower()

    # remove edition patterns
    text = re.sub(r"\b\d+(st|nd|rd|th)\s+edition\b", "", text)
    text = re.sub(r"\bedition\b", "", text)
    text = re.sub(r"\bed\b\.?", "", text)
    text = re.sub(r"\brevised\b", "", text)
    text = re.sub(r"\bupdated\b", "", text)

    # remove brackets content
    text = re.sub(r"\(.*?\)", "", text)

    # remove symbols
    text = re.sub(r"[^a-z0-9 ]", " ", text)

    # normalize spaces
    text = re.sub(r"\s+", " ", text)

    return text.strip()

## Clean book title function

def clean_book_title(title):
    if pd.isna(title):
        return None

    title = title.lower()
    title = title.split("/")[0]

    # remove editions
    title = re.sub(r"\b\d+(st|nd|rd|th)?\s*edition\b", "", title)
    title = re.sub(r"\bedition\b|\bed\b\.?", "", title)

    # remove years
    title = re.sub(r"\b(19|20)\d{2}\b", "", title)

    # Remove Colons and everything after
    title = re.sub(r":.*$", "", title)

    # keep + & :
    title = re.sub(r"[^a-z0-9+& ]", " ", title)
    title = re.sub(r"\s+", " ", title)
    print(title)
    return title.strip()


## Clean author name function

def clean_author_name(author):
    if not author or pd.isna(author):
        return None

    author = str(author).lower().strip()

    # 1️⃣ remove anything after slash (often title leakage)
    author = author.split("/")[0]

    # 2️⃣ remove editor-related words
    author = re.sub(
        r"\b(editor|editors|edited by|eds?)\b",
        "",
        author
    )

    # 3️⃣ remove years and numeric ranges (e.g. 1965-, 1999)
    author = re.sub(r"\b(18|19|20)\d{2}\b", "", author)
    author = re.sub(r"\d+", "", author)

    # 4️⃣ remove content inside brackets
    author = re.sub(r"\(.*?\)", "", author)

    # 5️⃣ handle "last, first" → "first last"
    if "," in author:
        parts = [p.strip() for p in author.split(",") if p.strip()]
        if len(parts) >= 2:
            author = " ".join(parts[1:] + [parts[0]])

    # 6️⃣ normalize hyphens and dots
    author = author.replace("-", " ")
    author = author.replace(".", " ")

    # 7️⃣ keep letters and spaces only
    author = re.sub(r"[^a-z ]", " ", author)

    # 8️⃣ normalize spaces
    author = re.sub(r"\s+", " ", author)

    return author.strip()



# =========================
# SCRAPING CORE
# =========================
def extract_description(soup):
    # Primary description selectors
    selectors = [
        "div.book-description-content",
        "div.work-description",
        "div.read-more__content"
    ]

    # 1️⃣ Try real description first
    for sel in selectors:
        tag = soup.select_one(sel)
        if not tag:
            continue

        text = tag.get_text(" ", strip=True)

        # reject Open Library placeholder
        if "doesn't have a description yet" in text.lower():
            continue

        # reject junk
        if len(text) < 10:
            continue

        return text

    # 2️⃣ FALLBACK: h2.work-subtitle
    subtitle = soup.select_one("h2.work-subtitle")
    if subtitle:
        subtitle_text = subtitle.get_text(" ", strip=True)

        if subtitle_text and len(subtitle_text) > 5:
            return subtitle_text

    return None



def fetch_by_isbn_html(isbn):
    url = f"https://openlibrary.org/isbn/{isbn}"
    try:
        r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
        if r.status_code != 200:
            return None

        soup = BeautifulSoup(r.text, "html.parser")

        # STRICT ISBN CHECK
        page_text = soup.get_text()
        if isbn not in page_text:
            return None

        return extract_description(soup)
    except:
        return None


def fetch_by_title_author_html(title, author):
    try:
        query = f"{title} {author}"

        r = requests.get(
            "https://openlibrary.org/search",
            params={"q": query},
            headers=HEADERS,
            timeout=TIMEOUT
        )

        if r.status_code != 200:
            return None

        soup = BeautifulSoup(r.text, "html.parser")

        # ✅ correct selector
        results = soup.select("li.searchResultItem a")

        if not results:
            return None

        # check first 5 results (edition ignored)
        for a in results[:5]:
            href = a.get("href", "")
            if not (href.startswith("/works/") or href.startswith("/books/")):
                continue


            work_url = "https://openlibrary.org" + href

            work_resp = requests.get(work_url, headers=HEADERS, timeout=TIMEOUT)
            if work_resp.status_code != 200:
                continue

            work_soup = BeautifulSoup(work_resp.text, "html.parser")
            desc = extract_description(work_soup)

            if desc:
                return desc

    except Exception as e:
        print("SEARCH ERROR:", e)

    return None


# =========================
# LOAD DATA
# =========================

for col in [DESC_COL, SOURCE_COL]:
    if col not in df.columns:
        df[col] = ""

# =========================
# MAIN PIPELINE
# =========================
for idx in tqdm(range(len(df)), desc="HTML scraping OpenLibrary"):
    if df.at[idx, DESC_COL]:
        continue

    raw_isbn = df.at[idx, ISBN_COL]
    raw_title = df.at[idx, TITLE_COL]
    raw_author = df.at[idx, AUTHOR_COL]

    isbn = clean_isbn(raw_isbn)
    title = clean_book_title(raw_title)
    author = clean_author_name(raw_author)

    description = None

    # 1️⃣ ISBN FIRST (STRICT)
    if isbn:
        description = fetch_by_isbn_html(isbn)
        if description:
            df.at[idx, DESC_COL] = description
            df.at[idx, SOURCE_COL] = "OpenLibrary"

    # 2️⃣ TITLE + AUTHOR FALLBACK
    if not description and title and author:
        description = fetch_by_title_author_html(title, author)
        print("Title+Author:" , description)
        if description:
            df.at[idx, DESC_COL] = description
            df.at[idx, SOURCE_COL] = "OpenLibrary"

    # 3️⃣ NOT FOUND
    if not description:
        df.at[idx, DESC_COL] = "Not Found"
        df.at[idx, SOURCE_COL] = "Not Found"

    if idx % SAVE_EVERY == 0:
        df.to_csv(OUTPUT_CSV, index=False)

    time.sleep(SLEEP_TIME)

df.to_csv(OUTPUT_CSV, index=False)
print("✅ Finished scraping with strict ISBN → Title+Author logic.")


HTML scraping OpenLibrary:   0%|          | 0/32012 [00:00<?, ?it/s]

network design 


HTML scraping OpenLibrary:   0%|          | 1/32012 [00:03<28:22:26,  3.19s/it]

multimedia information analysis and retrieval 


HTML scraping OpenLibrary:   0%|          | 2/32012 [00:05<23:43:25,  2.67s/it]

...

cinema today


HTML scraping OpenLibrary:  31%|███       | 9916/32012 [15:16:43<25:15:12,  4.11s/it]

cinematography 
Title+Author: None


HTML scraping OpenLibrary:  31%|███       | 9917/32012 [15:16:45<21:30:53,  3.51s/it]

dada and surrealism
